In [ ]:
import veloxchem as vlx
import adcc
from mpi4py import MPI
import sys
import numpy
from scipy.sparse import linalg

numpy.set_printoptions(precision=7, suppress=True)

### Prepare a VeloxChem SCF calculation

In [ ]:
# Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

#molecule_string = """
#    H 0 0 0
#    H 0 0 1.795239827225189
#    """

basis_set_label = 'STO-3G'
scf_settings = {'conv_thresh': 1.0e-6}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-4, 'nstates': 3}

In [ ]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [ ]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

In [ ]:
# MockTask class needed to run adcc with vlx
class MockTask:
    def __init__(self, mol, basis, comm, ostream):

        self.molecule = mol
        self.ao_basis = basis
        self.mpi_comm = comm
        self.ostream = ostream

In [ ]:
# SCF
task = MockTask(molecule, basis, comm, ostream)
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
scfdrv.task = task

In [ ]:
# SCF first-order properties
scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

### Excited-State Calculation via CIS (TDHF/TDA)

In [ ]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

In [ ]:
# Get MO coefficients

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

In [ ]:
# Get the overlap and Fock matrices

print(scfdrv.scf_tensors.keys())
fock = scfdrv.scf_tensors['F'][0]
ovlp = scfdrv.scf_tensors['S']
# Transform them from AO to MO basis
print(numpy.matmul(mo.T, numpy.matmul(ovlp, mo)))
print(numpy.matmul(mo.T, numpy.matmul(fock, mo)))

In [ ]:
# Get the CIS eigenvalues and eigenvectors

tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

### Calculate one- and two-particle density matrices

\begin{align}
    \gamma_{ij} &= - \sum_{a} x_{ia} x_{ja} \\
    \gamma_{ab} &= + \sum_{i} x_{ia} x_{ib} \\
    \Gamma_{iajb} &= - x_{ib} x_{ja}
\end{align}

In [ ]:
# VeloxChem unrelaxed one-particle density matrix and two-particle density matrix

vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

print(vlx_dm_oo)

### Transform the one-particle density matrices from the MO to the AO basis

\begin{equation}
    \gamma_{\mu \nu} = \sum_{pq} C_{\mu p} \gamma_{pq} C_{\nu q}
\end{equation}

In [ ]:
# Transform the one-particle density matrices from MO to AO basis
vlx_dm_oo_in_ao = numpy.matmul(mo_occ, numpy.matmul(vlx_dm_oo, mo_occ.T))
adcc_dm_oo_in_ao = 2.0*numpy.matmul(mo_occ, numpy.matmul(adcc_dm_oo[:nocc,:nocc], mo_occ.T))

vlx_dm_vv_in_ao = numpy.matmul(mo_vir, numpy.matmul(vlx_dm_vv, mo_vir.T))
adcc_dm_vv_in_ao = 2.0*numpy.matmul(mo_vir, numpy.matmul(adcc_dm_vv[:nvir,:nvir], mo_vir.T))

vlx_dm_ao = vlx_dm_oo_in_ao + vlx_dm_vv_in_ao
adcc_dm_ao = adcc_dm_oo_in_ao + adcc_dm_vv_in_ao

print("VLX:")
print(vlx_dm_ao)
print("\nADCC:")
print(adcc_dm_ao)

#### Transform the excitation vectors to the AO basis
\begin{equation}
    x_{\mu \nu} = \sum_{ia} C_{\mu i} x_{ia} C_{\nu a}
\end{equation}

In [ ]:
# Transform the transition density matrix (the excitation vectors) from MO to AO basis
# (will be needed for the two-particle density matrix)

tda_eig_vec_ao = numpy.matmul(mo_occ, numpy.matmul(tda_eig_vec_as_mat, mo_vir.T))
print(tda_eig_vec_ao)

In [ ]:
# Get the ERI tensor in AO basis (chemists' notation)

nao = mo.shape[0]

pqrs = numpy.zeros((nao, nao, nao, nao))
eri_drv = vlx.ElectronRepulsionIntegralsDriver(comm)
eri_drv.compute_in_mem(molecule, basis, pqrs)

The MO to AO transformation of the contribution of the two-particle density matrix with the ERI tensor is given as
\begin{align}
\begin{split}
    - \sum_{\sigma \sigma'} \sum_{ijab} x_{i_{\sigma} b_{\sigma}} x_{j_{\sigma'} a_{\sigma'}}
        \langle i_{\sigma} a_{\sigma'} || j_{\sigma'} b_{\sigma} \rangle
    &= - \sum_{ijab} x_{ib} x_{ja} [ 2 (ij|ab) - 4 (ib|aj) ] \\
    &= + \sum_{ijab} x_{ib} x_{ja} [ 4 (ib|aj) - 2 (ij|ab)  ] \\
    &= \sum_{ijab} x_{ib} x_{ja} \sum_{\mu \nu \theta \varphi}
        C_{\mu i} C_{\nu b} C_{\theta a} C_{\varphi j}  [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ] \\
    &= \sum_{\mu \nu \theta \varphi} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
        \sum_{ib} C_{\mu i} x_{ib} C_{\nu b} \sum_{ja} C_{\varphi j} x_{ja} C_{\theta a} \\
    &= \sum_{\mu \nu \theta \varphi} x_{\mu \nu} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]       \\
    &= \sum_{\mu \nu} x_{\mu \nu} \sum_{\theta \varphi} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
\end{split}
\end{align}

# Orbital response

\begin{equation}
   -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big) + \lambda_{ia} ( \varepsilon_i - \varepsilon_a ) = 
   R_{ia}
   %R^{\text{CIS}}_{ia}
\end{equation}

## Right-hand side

### 1. Contributions from the 1PDMs
\begin{align}
F^\gamma_{ia} &= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu,\sigma}\gamma^\sigma_{\mu\nu}(\mu\nu|\theta \varphi)-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu}\gamma^{\alpha}_{\mu\nu}(\mu\varphi|\theta\nu)\\
&= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a} 2F^{1,\alpha}_{\varphi\theta}-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{2,\alpha}_{\varphi\theta}\\
&= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F_{\varphi\theta}
\end{align}

In [ ]:
# Calculate two different parts and transform the sum to MO

F1_vlx = numpy.einsum('mn,mntp->pt', vlx_dm_ao, pqrs)
F2_vlx = numpy.einsum('mn,mptn->pt', vlx_dm_ao, pqrs)
# The density matrices already have a factor of 2 (for alpρφha and beta spin)
# hence a factor of 0.5 where the other spin part actually vanishes
F_1pdm_vlx = F1_vlx - 0.5*F2_vlx
vlx_1pdm_rhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, F_1pdm_vlx))
print(vlx_1pdm_rhs)

##### Using AODensityMatrix and LinearSolver objects and comp_lr_fock function

In [ ]:
# Use AODensityMatrix object to compute a Fock-like matrix
# expect something like F_{\varphi\theta}, equation above
# see linearsolver.py; we have to create a linear solver object:

from veloxchem.linearsolver import LinearSolver
from veloxchem.veloxchemlib import AOFockMatrix
from veloxchem.veloxchemlib import AODensityMatrix
from veloxchem.veloxchemlib import denmat

ao_density_object = AODensityMatrix([vlx_dm_ao], denmat.rest)

In [ ]:
linear_solver_object = LinearSolver(comm, ostream)
eri_dict = linear_solver_object.init_eri(molecule, basis)
# DFT information
dft_dict = linear_solver_object.init_dft(molecule, scfdrv.scf_tensors)
# PE information
pe_dict = linear_solver_object.init_pe(molecule, basis)
timing_dict = {}
fock_matrix_rhs_1pdm = AOFockMatrix(ao_density_object)
#print(fock_matrix_rhs_1pdm)
linear_solver_object.comp_lr_fock(fock_matrix_rhs_1pdm, ao_density_object, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)
print("Fock matrix from 1PDM, from LinearSolver:")
print(fock_matrix_rhs_1pdm)
print()
print("Fock matrix from 1PDM, using numpy.einsum:")
print(2*F_1pdm_vlx)
#We have to take 0.5*fock_matrix_rhs_1pdm because there is a sqrt(2) in the excitation vectors
#(normalized? alpha+beta?, but we are looking only at the alpha-alpha block of the multipliers) 
rhs_1pdm_from_LinearSolver = numpy.einsum('pi,pa->ia', mo_occ,
                            numpy.einsum('ta,pt->pa', mo_vir, 0.5*fock_matrix_rhs_1pdm.alpha_to_numpy(0)))
print("\nIn MO basis, 1PDM contribution to the RHS, LinearSolver:")
print(rhs_1pdm_from_LinearSolver)
print()
print("Original, for comparison:")
print(vlx_1pdm_rhs)

### 2. Contributions from the 2PDMs

\begin{align}
F^\Gamma_{ia} &= ~~ \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\theta,\nu,\sigma} x^\sigma_{\theta\nu}(\mu\varphi|\theta\nu) \Big] \\
&~~~ -\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[ \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi)-\sum_{\rho,\varphi} S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\theta,\nu,\sigma}x^\sigma_{\nu\theta}(\zeta\varphi|\theta\nu) \Big] \\
&= - \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\Big[ 2\sum_{\theta,\nu} x^\alpha_{\theta\nu}(\mu\varphi|\theta\nu)- \sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi) \Big] \\
&~~~~ +\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\Big[ 2\sum_{\theta,\nu} x^\alpha_{\nu\theta}(\zeta\varphi|\theta\nu)- \sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi) \Big] \\
&= - \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\Big[ \sum_{\theta,\nu} x^\alpha_{\theta\nu} \big[ 2 (\mu\varphi|\theta\nu)- (\mu\nu|\theta\varphi) \big] \Big] \\
&~~~~ +\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\Big[ \sum_{\theta,\nu} x^\alpha_{\nu\theta} \big[ 2(\zeta\varphi|\theta\nu)- (\zeta\nu|\theta\varphi) \big] \Big]
\end{align}

In [ ]:
# Care needs to be taken with the index order of the excitation vectors
# Calculate in different summation orders...

# 1)
FG1 = ( numpy.einsum('rz,mr->mz', ovlp,
                    numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                        (0.5*numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )
FG2 = ( numpy.einsum('mr,rz->mz', ovlp,
                    numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                        (0.5*numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )

# 2)
FG1_try2 =( numpy.einsum('rz,mr->mz', ovlp,
                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG2_try2 =( numpy.einsum('rz,mr->mz', ovlp,
                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                                    numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG3_try2 =( numpy.einsum('mr,rz->mz', ovlp,
                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                                    numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG4_try2 =( numpy.einsum('mr,rz->mz', ovlp,
                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                                    numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )

FG1_vlx =( numpy.einsum('rz,zm->rm', ovlp,
                       numpy.einsum('zp,mp->zm', tda_eig_vec_ao,
                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG1_vlx_ovlp =( numpy.einsum('rp,mp->rm',
                numpy.einsum('rz,zp->rp', ovlp, tda_eig_vec_ao),
                numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
            )

FG2_vlx =  numpy.einsum('rz,zm->rm', ovlp,
                       numpy.einsum('zn,nm->zm', tda_eig_vec_ao,
                                    numpy.einsum('tp,mntp->nm',tda_eig_vec_ao,pqrs)
                                   )
                      )

# Sum up terms accounting for spin factors
F_2pdm_vlx_A = 0.5*FG1_try2 - FG2_try2 
F_2pdm_vlx_B =-0.5*FG3_try2 + FG4_try2

F_2pdm_vlx = F_2pdm_vlx_A + F_2pdm_vlx_B
#print(F_2pdm_vlx)
# 3)
another_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_A))
                +numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_B))
               )
vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx)))
final_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, FG1-FG2)))
print(vlx_2pdm_rhs)
print()
print(another_vlx_2pdm_rhs)         
print()
print(final_vlx_2pdm_rhs)

In [ ]:
# 2PDM contribution, using AODensityMatrix, AOFockMatrix, LinearSolver
# we need the transpose of the vector because of how the exchange part is calculated
# (mn|tp)!=(mt|np), but (mn|tp)=(nm|tp)=(mn|pt)=(nm|pt) real orbitals

ao_density_2pdm_1 = AODensityMatrix([tda_eig_vec_ao.T], denmat.rest)
ao_density_2pdm_2 = AODensityMatrix([tda_eig_vec_ao], denmat.rest)
ao_fock_2pdm_1 = AOFockMatrix(ao_density_2pdm_1)
ao_fock_2pdm_2 =AOFockMatrix(ao_density_2pdm_2)
linear_solver_2pdm_1 = LinearSolver(comm, ostream) #Can we use the same object?
linear_solver_2pdm_1.comp_lr_fock(ao_fock_2pdm_1, ao_density_2pdm_1, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)
linear_solver_2pdm_2 = LinearSolver(comm, ostream) #Can we use the same object?
linear_solver_2pdm_2.comp_lr_fock(ao_fock_2pdm_2, ao_density_2pdm_2, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)
                        
fock_1 = (1.0*numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
        -2.0*numpy.einsum('mn,mnpt->pt',tda_eig_vec_ao,pqrs))

fock_2 = (1.0*numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
        -2.0*numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs))

print("What we are after, 1st term:")
print(fock_1)
print()
print("What we get from comp_lr_fock using tda_eig_vec_ao.T:")
print(ao_fock_2pdm_1)
print()
print("What we are after, 2nd term:")
print(fock_2)
print()
print("What we get from comp_lr_fock using tda_eig_vec_ao:")
print(ao_fock_2pdm_2)
print()
LR_2pdm_F1 =  numpy.einsum('rz,mr->mz', ovlp,
             numpy.einsum('pr,mp->mr', tda_eig_vec_ao, 0.5*ao_fock_2pdm_1.alpha_to_numpy(0)))
LR_2pdm_F2 =  numpy.einsum('mr,rz->mz', ovlp,
                    numpy.einsum('rp,zp->rz', tda_eig_vec_ao, 0.5*ao_fock_2pdm_2.alpha_to_numpy(0)))

rhs_2pdm_from_LinearSolver = numpy.einsum('mi,ma->ia', mo_occ, 
                                          numpy.einsum('za,mz->ma', mo_vir, LR_2pdm_F2-LR_2pdm_F1))
print("2PDM RHS using LinearSolver:")
print(rhs_2pdm_from_LinearSolver)
print()
print("2PDM RHS using numpy.einsum:")
print(final_vlx_2pdm_rhs)

## Left-hand side
### Initial guess for Lagrange multipliers $\boldsymbol{\lambda}$

\begin{equation}
    \lambda_{ia}^{(0)} = \frac{R_{ia}}{\varepsilon_i - \varepsilon_a}
\end{equation}

In [ ]:
# Neglect the non-diagonal part of the LHS,
# i.e. divide the RHS by the orbital-energy difference

# TODO: Avoid the for-loops, use something analogous to the mp2driver

vlx_rhs = final_vlx_2pdm_rhs + vlx_1pdm_rhs
lambda_guess = numpy.zeros((nocc,nvir))
mo_energies = scfdrv.scf_tensors['E']
#print(mo_energies[nocc:])

for i in range(nocc):
    ei=mo_energies[i]
    for a in range(nvir):
        ea=mo_energies[nocc+a]
        lambda_guess[i,a] = vlx_rhs[i,a]/(ei-ea)
        
print(lambda_guess)

In [ ]:
# Transform the initial guess for lambda to the AO basis

lambda_guess_ao = numpy.matmul(mo_occ, numpy.matmul(lambda_guess, mo_vir.T))
print(lambda_guess_ao)

### Contraction of the initial guess for $\boldsymbol{\lambda}$ with two-electron integrals

using $p=j, q=b$ as well as $p=b, q=j$ with the condition $\lambda_{pq} = \lambda_{qp}$ and assuming real orbitals, the equation is obtained as
\begin{align}
-\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big)
    &= -\sum_{\varphi \theta} C_{\varphi i} C_{\theta a} \sum_{\mu \nu}
        \lambda_{\mu \nu} \big[ 4 (\mu \nu | \varphi \theta)
        - (\mu \theta | \varphi \nu) - (\nu \theta | \varphi \mu) \big] \\
    %+ (\varepsilon_a - \varepsilon_i ) \gamma_{ia}
    &= - \sum_{\varphi \theta} \big[ C_{\varphi i} C_{\theta a} + C_{\theta i} C_{\varphi a} \big]
        \sum_{\mu \nu} \lambda_{\mu \nu} \big[ 2 (\mu \nu | \varphi \theta) - (\mu \theta | \varphi \nu) \big]
\end{align}

In [ ]:
# Contract with the ERI tensor
vlx_lhs1 = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
vlx_lhs2 = numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
vlx_lhs3 = numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)

vlx_lhs_ao = -4*vlx_lhs1 + vlx_lhs2 + vlx_lhs3

# Transform to MO
vlx_lhs_mo = numpy.matmul(mo_occ.T, numpy.matmul(vlx_lhs_ao, mo_vir))
#vlx_lhs_mo2 = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao))
print(vlx_lhs_mo)
#print(vlx_lhs_mo2)

In [ ]:
#Lambda guess contracted with eri using AODensityMatrix, AOFockMatrix, LinearSolver

ao_density_lambda = AODensityMatrix([lambda_guess_ao], denmat.rest)
ao_fock_lambda = AOFockMatrix(ao_density_lambda)
linear_solver_lambda = LinearSolver(comm, ostream) #Can we use the same object?
linear_solver_lambda.comp_lr_fock(ao_fock_lambda, ao_density_lambda, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)

L1_vlx = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
L2_vlx = numpy.einsum('mn,mptn->pt', lambda_guess_ao, pqrs)
L_total_vlx = 2*L1_vlx - L2_vlx
print("Lambda in AO using AOFockMatrix:")
print(ao_fock_lambda)
print()
print("Lambda in AO using numpy.einsum:")
print(L_total_vlx)
fock_lambda_numpy = ao_fock_lambda.alpha_to_numpy(0)
#print()
#print(fock_lambda_numpy)
lambda_mo_from_fock = ( numpy.matmul(mo_occ.T, numpy.matmul(fock_lambda_numpy, mo_vir)) 
                       + numpy.matmul(mo_vir.T, numpy.matmul(fock_lambda_numpy, mo_occ)).T )
print()
print("In MO, using the Fock Matrix object")
print(lambda_mo_from_fock)
print()
print("In MO, from the Equation:")
print(vlx_lhs_mo)

In [ ]:
# LHS and guess
print("vlx_lhs_mo:")
print(vlx_lhs_mo)
print()
print("lambda_guess:")
print(lambda_guess)

#### Full LHS 
\begin{equation}
   -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big) + \lambda_{ia} ( \varepsilon_i - \varepsilon_a )
\end{equation}

In [ ]:
# Add product of guess and orbital-energy differences to get full LHS

# TODO: Again, avoid for-loops

vlx_full_lhs = numpy.zeros((nocc,nvir))
for i in range(nocc):
    ei = mo_energies[i]
    for a in range(nvir):
        ea = mo_energies[nocc+a]
        vlx_full_lhs[i,a] = vlx_lhs_mo[i,a] + lambda_guess[i,a]*(ei-ea)

vlx_full_lhs_vec = vlx_full_lhs.reshape(nocc*nvir)
vlx_rhs_vec = vlx_rhs.reshape(nocc*nvir)

print(vlx_full_lhs_vec)
print()
print(vlx_rhs_vec)

## Solution via Conjugate Gradient

To solve: $\mathbf{A} \mathbf{x} = \mathbf{b}$

However, we do not "know" the matrix $\mathbf{A}$, only the matrix-vector product $\mathbf{Ax}$.
Thus, define it as a linear operator with a vector $\mathbf{x}$ as an argument.

### Defining a linear operator

In [ ]:
# Assuming that all other variables (nocc, nvir, MOs, ERIs) are known from the "global" scope
def Ax(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix. Transforms guess to AO basis, carries out
    matrix-vector product and transforms back to MO.
    
    v: lambda at current iteration"""

    current_lambda = v.reshape(nocc,nvir)
    print("VLX INPUT:")
    print(current_lambda)
    print()
    
    # Transform to AO
    vlx_lambda_ao = numpy.matmul(mo_occ, numpy.matmul(current_lambda, mo_vir.T))
 
    # Calculate the different contractions
    vlx_lhs_interm = (-4*numpy.einsum('mn,mntp->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,mtpn->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,ntpm->pt', vlx_lambda_ao, pqrs)
                      )
    # Transform back to MO
    vlx_1pdm_lhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_interm))

# Add the diagonal part
    for i in range(nocc):
        ei = mo_energies[i]
        for a in range(nvir):
            ea = mo_energies[nocc+a]
            vlx_1pdm_lhs[i,a] += current_lambda[i,a]*(ei-ea) 
    print("VLX OUTPUT:")    
    print(vlx_1pdm_lhs)
    print()
    return vlx_1pdm_lhs.reshape(nocc*nvir)

In [ ]:
def Ax_with_LinearSolver(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix, using AODensityMatrix, AOFockMatrix, and comp_lr_fock
    Transforms guess to AO basis, carries out
    matrix-vector product and transforms back to MO.
    
    v: lambda at current iteration"""
    
    current_lambda = v.reshape(nocc,nvir)
    #print("LinearSolver VLX INPUT:")
    #print(current_lambda)
    #print()

    # Transform to AO
    vlx_lambda_ao = numpy.matmul(mo_occ, numpy.matmul(current_lambda, mo_vir.T))
    
    #Create AODensityMatrix object from lambda in AO
    vlx_ao_density_lambda = AODensityMatrix([vlx_lambda_ao], denmat.rest)
    
    #Create a Fock Matrix Object (initialized with zeros)
    vlx_fock_lambda = AOFockMatrix(vlx_ao_density_lambda)
    
    #Create linear solver object and compute the Fock matrix (contracts AODensityMatrix with ERI: 2J-K)
    linear_solver_lambda = LinearSolver(comm, ostream) #Can we use the same object?
    linear_solver_lambda.comp_lr_fock(vlx_fock_lambda, vlx_ao_density_lambda, molecule, basis,
                                      eri_dict, dft_dict, pe_dict, timing_dict)

    #Transform AOFockMatrix to numpy array (here we took only the alpha block)
    vlx_fock_lambda_numpy = vlx_fock_lambda.alpha_to_numpy(0)
    
    
    #Transform to MO basis (symmetrized w.r.t. occ. and virt.)
    vlx_lambda_mo_from_fock = -( numpy.matmul(mo_occ.T, numpy.matmul(vlx_fock_lambda_numpy, mo_vir)) 
                               + numpy.matmul(mo_vir.T, numpy.matmul(vlx_fock_lambda_numpy, mo_occ)).T )

    
    # Add the diagonal part
    for i in range(nocc):
        ei = mo_energies[i]
        for a in range(nvir):
            ea = mo_energies[nocc+a]
            vlx_lambda_mo_from_fock[i,a] += current_lambda[i,a]*(ei-ea) 
    #print("LinearSolver VLX OUTPUT:")    
    #print(vlx_lambda_mo_from_fock)
    #print()
    return vlx_lambda_mo_from_fock.reshape(nocc*nvir)

In [ ]:
# Solve via CG

vlx_guess_vec=lambda_guess.reshape(nocc*nvir)

A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=Ax)
solution, w = linalg.cg(A=A, b=vlx_rhs_vec, x0=vlx_guess_vec, tol=1e-8, maxiter=25)

In [ ]:
#Use Ax_with_LinearSolver for conjugate gradient
lr_vlx_guess_vec=lambda_guess.reshape(nocc*nvir)

#print("Run Ax_with_LinearSolver once, lambda_guess as input:")
#Ax_with_LinearSolver(lambda_guess.reshape(nocc*nvir))
#print()

#print("Create scipy.linalg LinearOperator:")
LR_A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=Ax_with_LinearSolver)

LR_full_RHS_1d_vec = (rhs_2pdm_from_LinearSolver + rhs_1pdm_from_LinearSolver).reshape(nocc*nvir)

print("Starting conjugate gradient...\n")
LR_solution, lr_w = linalg.cg(A=LR_A, b=LR_full_RHS_1d_vec, x0=LR_full_RHS_1d_vec.copy(), tol=1e-8, maxiter=25)

print("The solution in matrix form:")
print(LR_solution.reshape(nocc,nvir))
print()
print("The solution using numpy.einsum:")
print(solution.reshape(nocc,nvir))